In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# 1127 -Tinkoff

# 1673 - Тинькофф

# DATA LOAD

In [2]:
comment = pd.read_excel('data.xlsx', sheet_name = 'Comments', skiprows=1)
comment.head()

,Социальная сеть,ID поста,ID комментария,Владелец,ID Владелеца,Текст,Эмоциональный окрас,Дата,Лайков
0,Вконтакте,-115807015_2334,5412482_2336,Валентин Точилкин,5412482,Задумка понравилась! Думаю будет смотреться лу...,Нейтральность,2021-12-31 08:19:05,1
1,Вконтакте,-30666517_1774956,308730199_1774960,Денис Гончаров,308730199,Вам тоже здоровья и удачи в прошлом и крепкого...,Вежливость,2021-12-31 07:09:40,9
2,Вконтакте,-30666517_1774956,10784303_1774979,Олег Кирин,10784303,Полагаю: Вера и Надежда – это тимлид и прожект...,Юмор,2021-12-31 09:20:39,4
3,Вконтакте,-30666517_1774956,586953361_1775092,Замбек Замбеков,586953361,этот интеллект на али давно существует,Нейтральность,2021-12-31 20:38:00,1
4,Вконтакте,-30666517_1774956,19496621_1775350,Александр Бирабиджанов,19496621,Будет вечный Свет.,Нейтральность,2022-01-03 00:04:52,1


In [3]:
comment = comment.fillna('Пропуск')

# bigramm

In [4]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm
import numpy as np
import pymorphy2
import nltk
morph = pymorphy2.MorphAnalyzer()
stopwords_ru = stopwords.words("russian")

# 1 этап

In [5]:
data = comment.Текст.values.tolist()

# 2 этап

In [6]:
data = [re.sub('\s+', ' ', sent) for sent in data] # убираем new line characters

In [7]:
data = [re.sub("\d+", ' ', sent) for sent in data] # убираем всё цифры

In [8]:
data = [re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", ' ', sent) for sent in data] # убираем url

# Левая ветка

## 3 этап левая ветка

In [9]:
data_left = [re.sub(r'[^\w\s]','', sent) for sent in data] # убираем знаки препинания.

## 4.1 этап левая ветка (нормализация)

In [10]:
data_left_norm = []  # с нормализацией
for i in tqdm(range(len(data_left))):
    data_left_norm.append(list(morph.parse(token)[0].normal_form for token in word_tokenize(data_left[i].lower().strip(), language="russian")))

100%|███████████████████████████████████████████████████████████████████████████| 21498/21498 [00:53<00:00, 403.99it/s]


## 4.2 этап левая ветка

In [11]:
data_left_d = []        # без нормализации
for i in tqdm(range(len(data_left))):
    data_left_d.append(word_tokenize(data_left[i].lower().strip(), language="russian"))

100%|█████████████████████████████████████████████████████████████████████████| 21498/21498 [00:01<00:00, 10964.82it/s]


## 5.1 этап левая ветка (нормализация)

In [12]:
bi_gram_norm = []
for sentence in tqdm(data_left_norm): # по постам
    data_left_norm_stop = []
    for word in sentence:                # по словам
        if word in stopwords_ru:
            continue
        else:
            data_left_norm_stop.append(word)
    bi_gram_norm += list(nltk.bigrams(data_left_norm_stop))

100%|█████████████████████████████████████████████████████████████████████████| 21498/21498 [00:00<00:00, 53764.82it/s]


## 5.2 этап левая ветка

In [13]:
bi_gram = []
for sentence in tqdm(data_left_d): # по постам
    data_left_norm_stop = []
    for word in sentence:                # по словам
        if word in stopwords_ru:
            continue
        else:
            data_left_norm_stop.append(word)
    bi_gram += list(nltk.bigrams(data_left_norm_stop))

100%|█████████████████████████████████████████████████████████████████████████| 21498/21498 [00:00<00:00, 54415.04it/s]


## 6.1 этап левая ветка (нормализация)

In [14]:
bgfd_norm = nltk.FreqDist(bi_gram_norm) # нормализованы
bgfd_norm.most_common(10)

[(('бесплатный', 'обслуживание'), 594),
 (('обслуживание', 'навсегда'), 352),
 (('карта', 'тинькофф'), 266),
 (('дебетовый', 'карта'), 243),
 (('тинькофф', 'блэк'), 242),
 (('кредитный', 'карта'), 212),
 (('карта', 'tinkoff'), 179),
 (('ссылка', 'получить'), 172),
 (('подписка', 'тинькофф'), 161),
 (('получить', 'бонус'), 159)]

## 6.2 этап левая ветка

In [15]:
bgfd = nltk.FreqDist(bi_gram) # не нормализованы
bgfd.most_common(10)

[(('бесплатное', 'обслуживание'), 295),
 (('моей', 'ссылке'), 283),
 (('бесплатным', 'обслуживанием'), 271),
 (('тинькофф', 'блэк'), 242),
 (('обслуживанием', 'навсегда'), 230),
 (('tinkoff', 'black'), 149),
 (('блэк', 'бесплатным'), 146),
 (('добрый', 'день'), 137),
 (('бесплатно', 'месяца'), 134),
 (('club', 'тинькофф'), 131)]

# Правая ветка

## 3 этап правая ветка

In [24]:
data_right = ' '.join(data)
data_right = sent_tokenize(data_right, language="russian")  # по предложениям

In [28]:
data_right = [re.sub(r'[^\w\s]','', sent) for sent in data_right] # убираем знаки препинания.

## 4.1 этап правая ветка (нормализация)

In [29]:
data_right_norm = []  # с нормализацией
for i in tqdm(range(len(data_right))):
    data_right_norm.append(list(morph.parse(token)[0].normal_form for token in word_tokenize(data_right[i].lower().strip(), language="russian")))

100%|███████████████████████████████████████████████████████████████████████████| 22954/22954 [00:53<00:00, 431.90it/s]


## 4.2 этап правая ветка

In [30]:
data_right_d = []        # без нормализации
for i in tqdm(range(len(data_right))):
    data_right_d.append(word_tokenize(data_right[i].lower().strip(), language="russian"))

100%|█████████████████████████████████████████████████████████████████████████| 22954/22954 [00:02<00:00, 10852.04it/s]


## 5.1 этап правая ветка (нормализация)

In [31]:
bi_gram_norm = []
for sentence in tqdm(data_right_norm): # по предложениям
    data_right_norm_stop = []
    for word in sentence:                # по словам
        if word in stopwords_ru:
            continue
        else:
            data_right_norm_stop.append(word)
    bi_gram_norm += list(nltk.bigrams(data_right_norm_stop))

100%|█████████████████████████████████████████████████████████████████████████| 22954/22954 [00:00<00:00, 56258.26it/s]


## 5.2 этап правая ветка

In [32]:
bi_gram = []
for sentence in tqdm(data_right_d): # по постам
    data_right_norm_stop = []
    for word in sentence:                # по словам
        if word in stopwords_ru:
            continue
        else:
            data_right_norm_stop.append(word)
    bi_gram += list(nltk.bigrams(data_right_norm_stop))

100%|█████████████████████████████████████████████████████████████████████████| 22954/22954 [00:00<00:00, 55839.04it/s]


## 6.1 этап правая ветка (нормализация)

In [33]:
bgfd_norm = nltk.FreqDist(bi_gram_norm) # нормализованы
bgfd_norm.most_common(10)

[(('бесплатный', 'обслуживание'), 594),
 (('обслуживание', 'навсегда'), 351),
 (('карта', 'тинькофф'), 268),
 (('дебетовый', 'карта'), 243),
 (('тинькофф', 'блэк'), 242),
 (('кредитный', 'карта'), 212),
 (('карта', 'tinkoff'), 179),
 (('ссылка', 'получить'), 172),
 (('подписка', 'тинькофф'), 161),
 (('получить', 'бонус'), 159)]

## 6.2 этап правая ветка

In [34]:
bgfd = nltk.FreqDist(bi_gram) # не нормализованы
bgfd.most_common(10)

[(('бесплатное', 'обслуживание'), 295),
 (('моей', 'ссылке'), 283),
 (('бесплатным', 'обслуживанием'), 271),
 (('тинькофф', 'блэк'), 242),
 (('обслуживанием', 'навсегда'), 229),
 (('tinkoff', 'black'), 149),
 (('блэк', 'бесплатным'), 146),
 (('добрый', 'день'), 137),
 (('club', 'тинькофф'), 131),
 (('месяца', 'подписки'), 129)]